In [1]:
import pickle
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd

In [2]:
def readData(datafile):
    datafileImport = "data/" + datafile
    infile = open(datafileImport, 'rb')
    new_dict = pickle.load(infile)
    return new_dict

In [3]:
data=["TY_Koc_Sales.pkl","TY_Koc_Basket.pkl","TY_Koc_Fav.pkl","TY_Koc_Gender.pkl","TY_Koc_Impression.pkl"\
      ,"TY_Koc_Price.pkl","TY_Koc_Quantity_Demand.pkl","TY_Koc_Quantity.pkl","TY_Koc_Rating_Review.pkl"\
      ,"TY_Koc_Removefromfav.pkl","TY_Koc_Size_Atts.pkl","TY_Koc_Visit.pkl"]

In [4]:
datas={"salesData":[],
       "basket":[],
       "fav":[],
       "gender":[],
       "impression":[],
       "price":[],
       "demand":[],
       "quantity":[],
       "rating":[],
       "removeFromFav":[],
       "sizeAtt":[],
       "visit":[]
       }

In [5]:
keys=list(datas.keys())
for i in range(len(data)):
    datas[keys[i]]=readData(data[i])

In [6]:
min_date = min(datas["salesData"]["order_date"])
max_date = max(datas["salesData"]["order_date"])
print("Avaliable min date = ", min_date, "\nAvaliable max date = ", max_date)
print("Total timeframe of the salesData",\
      (max(datas["salesData"]["order_date"])-min(datas["salesData"]["order_date"])).days+1)

Avaliable min date =  2021-04-01 
Avaliable max date =  2021-10-24
Total timeframe of the salesData 207


In [7]:
# Zero padding
sales= datas["salesData"].sort_values(by=['product_id','order_date'])
basket= datas["basket"].sort_values(by=['product_id','date'])
visit= datas["visit"].sort_values(by=['product_id','date'])
impression= datas["impression"].sort_values(by=['product_id','date'])
demand= datas["demand"].sort_values(by=['product_id','date'])
removeFromFav= datas["removeFromFav"].sort_values(by=['product_id','date'])

# Elongate
fav= datas["fav"].sort_values(by=['product_id','date'])
rating= datas["rating"].sort_values(by=['product_id','date'])
#Needs to be analyzed
price= datas["price"].sort_values(by=['product_id','created_date'])


# For clustering, no place in regression
attr = datas["sizeAtt"]
gender= datas["gender"]

# Discarded
stock= datas["quantity"].sort_values(by=['product_id','date'])

In [26]:
rating[(rating.product_id == 207940697241116)].head(50)

,product_id,date,reviewCount,rating
9234,207940697241116,2021-04-05,2,5.000000
21836,207940697241116,2021-04-06,1,1.000000
34722,207940697241116,2021-04-07,2,5.000000
47210,207940697241116,2021-04-08,5,4.200000
53926,207940697241116,2021-04-09,1,5.000000
95248,207940697241116,2021-04-10,4,4.000000
105050,207940697241116,2021-04-11,1,5.000000
130258,207940697241116,2021-04-13,2,5.000000
146090,207940697241116,2021-04-14,3,5.000000
155347,207940697241116,2021-04-15,32,4.937500


In [14]:
print("Number of uniqe products in each table")
for name,d in datas.items():
    print(name,len(Counter(d["product_id"]).keys())) # equals to list(set(words)))

#len(Counter(datas["salesData"]["product_id"]).values()) # counts the elements' frequency

Number of uniqe products in each table
salesData 68310
basket 126724
fav 145346
gender 327774
impression 78716
price 718986
demand 16275
quantity 1691
rating 32606
removeFromFav 127377
sizeAtt 959317
visit 310131


In [15]:
#Stock data for a specific product
stock = datas["quantity"]
stock[(stock.product_id == 2056551686182716)].sort_values(by=['date'])

,product_id,date,quantity
86846,2056551686182716,2021-04-01,29
70161,2056551686182716,2021-04-02,26
96608,2056551686182716,2021-04-03,22
94748,2056551686182716,2021-04-04,21
111154,2056551686182716,2021-04-05,20
...,...,...,...
20308,2056551686182716,2021-10-12,5
31971,2056551686182716,2021-10-13,1
0,2056551686182716,2021-10-14,0
82307,2056551686182716,2021-10-21,0


In [16]:
datas["salesData"][(datas["salesData"].product_id == 3557874004865180)].sort_values(by=['order_date'])

,product_id,brand_id,order_date,sales,current_bu_group_name,current_category_name
442376,3557874004865180,940277741,2021-05-03,1,Ev,Avize


In [34]:
datas["demand"][(datas["demand"].product_id == 2056551686182716)].sort_values(by=['date'])

,product_id,date,quantity_demand
95954,2056551686182716,2021-04-05,1
28433,2056551686182716,2021-04-11,1
160,2056551686182716,2021-04-21,1
46070,2056551686182716,2021-04-23,1
41440,2056551686182716,2021-04-24,1
31101,2056551686182716,2021-04-25,1
88916,2056551686182716,2021-04-26,1
78095,2056551686182716,2021-04-27,2
67007,2056551686182716,2021-04-29,1
56418,2056551686182716,2021-05-03,1


In [45]:
max(datas["salesData"]["order_date"])-min(datas["salesData"]["order_date"])
min(datas["fav"].sort_values(by=['date']).fav)

1

In [53]:
datas["impression"][datas["impression"].product_id == 3557874004865180].sort_values(by=['date']).head(50)

,product_id,date,impression
251292,3557874004865180,2021-04-01,10
156218,3557874004865180,2021-04-02,5
73154,3557874004865180,2021-04-03,14
801739,3557874004865180,2021-04-04,11
745268,3557874004865180,2021-04-05,1
434723,3557874004865180,2021-04-08,1
1259058,3557874004865180,2021-04-09,2
1618981,3557874004865180,2021-04-10,7
1586574,3557874004865180,2021-04-11,9
1485438,3557874004865180,2021-04-12,5


In [71]:
len(Counter(datas["price"][datas["price"].price == 0].sort_values(by=['created_date']).product_id).keys())


188743

In [70]:
datas["price"][datas["price"].price == 0].sort_values(by=['created_date'])

,product_id,price,created_date
107105,1622646028236011,0.0,2021-04-01
107412,4517481808258390,0.0,2021-04-01
107411,2952563082286010,0.0,2021-04-01
107410,1471962003079782,0.0,2021-04-01
107409,4800161062261114,0.0,2021-04-01
...,...,...,...
2429998,1077072651441408,0.0,2021-10-24
2429997,1229835078177613,0.0,2021-10-24
2429996,47595181075257,0.0,2021-10-24
2430002,3023489139003682,0.0,2021-10-24


In [57]:
datas["price"]

,product_id,price,created_date
0,4132725922387191,0.00,2021-07-25
1,1854209447749994,0.00,2021-07-25
2,3385201861738318,0.00,2021-07-25
3,4901072044437340,0.00,2021-07-25
4,4981538539553204,0.00,2021-07-25
...,...,...,...
2486350,2803183558627727,119.84,2021-04-07
2486351,2501867247806930,1214.69,2021-04-07
2486352,729286085459627,2813.38,2021-04-07
2486353,4995275069845093,247.93,2021-04-07


In [64]:
datas["salesData"][(datas["salesData"].product_id == 4132725922387191)].sort_values(by=['order_date'])

,product_id,brand_id,order_date,sales,current_bu_group_name,current_category_name


In [ ]:
import pickle
import numpy as np
import random


def readData(datafile):
    datafileImport="data/"+datafile
    #datafileExport="data2/"+datafile
    infile = open(datafileImport, 'rb')
    new_dict = pickle.load(infile)
    #found_dict=new_dict.loc[new_dict['product_id'].isin(random_ids)]
    #file = open(datafileExport, 'wb')
    #pickle.dump(found_dict, file)
    #file.close()
    return new_dict

salesData="TY_Koc_Sales.pkl"
basket="TY_Koc_Basket.pkl"
fav="TY_Koc_Fav.pkl"
gender="TY_Koc_Gender.pkl"
impression="TY_Koc_Impression.pkl"
price="TY_Koc_Price.pkl"
demand="TY_Koc_Quantity_Demand.pkl"
quantity="TY_Koc_Quantity.pkl"
rating="TY_Koc_Rating_Review.pkl"
removeFromFav="TY_Koc_Removefromfav.pkl"
sizeAtt="TY_Koc_Size_Atts.pkl"
visit="TY_Koc_Visit.pkl"
data=[salesData,basket,fav,gender,impression,price,demand,quantity,rating,removeFromFav,sizeAtt,visit]
datas={"salesData":[],
       "basket":[],
       "fav":[],
       "gender":[],
       "impression":[],
       "price":[],
       "demand":[],
       "quantity":[],
       "rating":[],
       "removeFromFav":[],
       "sizeAtt":[],
       "visit":[]
       }
keys=list(datas.keys())
for i in range(len(data)):
    datas[keys[i]]=readData(data[i])


dates = sorted(datas["salesData"]["order_date"].unique())
#productIDs= datas["salesData"]["product_id"].unique()

print(len(datas["price"]['product_id'].unique()))
sales= datas["salesData"].sort_values(by=['order_date'])
ids=sales["product_id"]
rules1Problems=[]
rules2Problems=[]
rules3Problems=[]
for productID in ids:
    #rule1
    """
    productDatas=datas["salesData"][datas["salesData"].product_id==productID]
    firstOrder=min(productDatas["order_date"])

    pricesData=datas["price"][datas["price"].product_id==productID]
    if len(pricesData)==0:
        #print(pricesData)
        #print(firstOrder, " ", productID)
        rules1Problems.append(productID)
        continue
    firstChange=min(pricesData["created_date"])
    if firstOrder<firstChange:
        print(firstOrder," ",firstChange, " ", productID)
        rules1Problems.append(productID)
    """
   #rule2
    """
    pricesData = datas["price"][datas["price"].product_id == productID]
    if len(pricesData) == 0:
        continue
    ChangeDates = pricesData["created_date"]
    curPrice=0
    for ChangeDate in ChangeDates:
        priceGivenID = pricesData[pricesData.created_date == ChangeDate]
        if curPrice==0:
            curPrice=priceGivenID['price'].iloc[0]
            continue
        elif priceGivenID['price'].iloc[0]==0:
            print(productID," ", curPrice, " ", priceGivenID['price'].iloc[0])
            rules2Problems.append(productID)
    """

    #rules3
    StartDate=min(datas["salesData"]['order_date'])
    EndDate=min(datas["salesData"]['order_date'])
    productDatas = datas["salesData"][datas["salesData"].product_id == productID]
    pricesData = datas["price"][datas["price"].product_id == productID]
    newGap=1
    if len(pricesData) == 0:
        continue
    ChangeDates = pricesData["created_date"]

    for ChangeDate in ChangeDates:
        priceGivenID = pricesData[pricesData.created_date == ChangeDate]
        curPrice = priceGivenID['price'].iloc[0]
        startPrice=-1
        if curPrice == 0:
            StartDate = ChangeDate
            startPrice = curPrice
            newGap=1
        else:
            EndDate = ChangeDate
            if newGap==1: # price 0 olduğu alandan + olduğu aralığa bakıyor - yeni bir 0 olmayan gelince bakmaya gerek yok.
                newGap=0
            else:
                continue
        productSalesData = productDatas[(productDatas.order_date>=StartDate) & (productDatas.order_date<EndDate)]
        if len(productSalesData)!=0:
            rules3Problems.append(productID)
            print("price of sales: ", startPrice,"sales Date ",ChangeDate)
print(rules3Problems)

Overall = {}

dates = sorted(datas["salesData"]["order_date"].unique())
productIDs= datas["salesData"]["product_id"].unique()
for productID in productIDs:
    for date in dates:
        Overall[productID]={"date":date}
        if date in datas["salesData"]['order_date']:
            Overall[productID]["sales"]=datas["salesData"][datas["salesData"].product_id==productID]['sales']
        else:
            Overall[productID]["sales"]=0

In [82]:
sales[100:150]

,product_id,brand_id,order_date,sales,current_bu_group_name,current_category_name
207064,68413811038,8169316966,2021-09-06,1,FMCG,Lokum
181333,68413811038,8169316966,2021-09-07,4,FMCG,Lokum
452706,68413811038,8169316966,2021-09-08,2,FMCG,Lokum
405501,68413811038,8169316966,2021-09-09,2,FMCG,Lokum
607307,68413811038,8169316966,2021-09-11,2,FMCG,Lokum
553942,68413811038,8169316966,2021-09-12,4,FMCG,Lokum
509023,68413811038,8169316966,2021-09-13,3,FMCG,Lokum
105919,68413811038,8169316966,2021-09-15,2,FMCG,Lokum
60840,68413811038,8169316966,2021-09-16,2,FMCG,Lokum
285807,68413811038,8169316966,2021-09-18,2,FMCG,Lokum


In [84]:
dates = sorted(datas["salesData"]["order_date"].unique())
dates

[datetime.date(2021, 4, 1),
 datetime.date(2021, 4, 2),
 datetime.date(2021, 4, 3),
 datetime.date(2021, 4, 4),
 datetime.date(2021, 4, 5),
 datetime.date(2021, 4, 6),
 datetime.date(2021, 4, 7),
 datetime.date(2021, 4, 8),
 datetime.date(2021, 4, 9),
 datetime.date(2021, 4, 10),
 datetime.date(2021, 4, 11),
 datetime.date(2021, 4, 12),
 datetime.date(2021, 4, 13),
 datetime.date(2021, 4, 14),
 datetime.date(2021, 4, 15),
 datetime.date(2021, 4, 16),
 datetime.date(2021, 4, 17),
 datetime.date(2021, 4, 18),
 datetime.date(2021, 4, 19),
 datetime.date(2021, 4, 20),
 datetime.date(2021, 4, 21),
 datetime.date(2021, 4, 22),
 datetime.date(2021, 4, 23),
 datetime.date(2021, 4, 24),
 datetime.date(2021, 4, 25),
 datetime.date(2021, 4, 26),
 datetime.date(2021, 4, 27),
 datetime.date(2021, 4, 28),
 datetime.date(2021, 4, 29),
 datetime.date(2021, 4, 30),
 datetime.date(2021, 5, 1),
 datetime.date(2021, 5, 2),
 datetime.date(2021, 5, 3),
 datetime.date(2021, 5, 4),
 datetime.date(2021, 5, 5),

In [97]:
def zero_padding(data_ls, date_ls):
    ls = [0 for i in range(len(dates))]
    for d in zip(data_ls, date_ls):
        index =  (d[1]-min_date).days
        ls[index] = d[0]
    return ls
   

In [98]:
import datetime
data_ls = [1, 2, 3]
date_ls = [datetime.date(2021, 4, 1),
 datetime.date(2021, 4, 2),
 datetime.date(2021, 4, 3)]
ls = zero_padding(data_ls, date_ls)


In [99]:
(datetime.date(2021, 4, 1)-min_date).days

0

In [100]:
ls

[1,
 2,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]